In [1]:
import os
import pandas as pd
import numpy as np
from utilities.common import check_nans, filter_df_by_ids, parse_score, get_gs_df
pd.set_option('display.max_columns', None)

## Load all data

In [2]:
PATH = "../data/raw/matches/atp_matches_"

atp_all = pd.read_csv("../data/raw/matches/atp_matches_1968.csv")

for year in range(1969, 2025):
    data = pd.read_csv(PATH + str(year) + ".csv")
    atp_all = pd.concat([atp_all, data], axis=0)


## Rafa, Novak, Roger first ATP matches

In [3]:
nadal_id = 104745
federer_id = 103819
djokovic_id = 104925

atp_all['tourney_date'] = pd.to_datetime(atp_all['tourney_date'], format='%Y%m%d')

nadal_first = atp_all[(atp_all['winner_id'] == nadal_id) | (atp_all['loser_id'] == nadal_id)].nsmallest(1, 'tourney_date')
federer_first = atp_all[(atp_all['winner_id'] == federer_id) | (atp_all['loser_id'] == federer_id)].nsmallest(1, 'tourney_date')
djokovic_first = atp_all[(atp_all['winner_id'] == djokovic_id) | (atp_all['loser_id'] == djokovic_id)].nsmallest(1, 'tourney_date')

print(f"Nadal: {nadal_first['tourney_date'].values[0]}" 
      f"Federer: {federer_first['tourney_date'].values[0]}" 
      f"Djokovic: {djokovic_first['tourney_date'].values[0]}")

Nadal: 2002-04-29T00:00:00.000000000Federer: 1998-07-06T00:00:00.000000000Djokovic: 2004-04-09T00:00:00.000000000


## Load 1998-2024

In [4]:
atp_recent = pd.read_csv("../data/raw/matches/atp_matches_1998.csv")

for year in range(1999, 2025):
    data = pd.read_csv(PATH + str(year) + ".csv")
    atp_recent = pd.concat([atp_recent, data], axis=0)
    
atp_recent['tourney_date'] = pd.to_datetime(atp_recent['tourney_date'], format='%Y%m%d', errors='coerce')

## Handle NaN values

In [5]:
check_nans(atp_recent)

Number of missing entries in 'surface': 53
Number of missing entries in 'winner_seed': 48212
Number of missing entries in 'winner_entry': 71442
Number of missing entries in 'winner_ht': 1612
Number of missing entries in 'winner_age': 5
Number of missing entries in 'loser_seed': 63172
Number of missing entries in 'loser_entry': 65076
Number of missing entries in 'loser_hand': 4
Number of missing entries in 'loser_ht': 3261
Number of missing entries in 'loser_age': 3
Number of missing entries in 'minutes': 9059
Number of missing entries in 'winner_rank': 703
Number of missing entries in 'winner_rank_points': 703
Number of missing entries in 'loser_rank': 1683
Number of missing entries in 'loser_rank_points': 1686
Number of rows with missing player statistics: 7278
Total rows with at least one missing value: 81831


In [6]:
gs_matches_2010_2024 = (get_gs_df(atp_recent))

winner_ids = gs_matches_2010_2024['winner_id']
loser_ids = gs_matches_2010_2024['loser_id']

all_unique_gs_players = pd.concat([winner_ids, loser_ids]).dropna().astype(int).unique()
all_unique_gs_players = sorted(all_unique_gs_players) 

print(f"Total unique players in Grand Slams since 2010: {len(all_unique_gs_players)}")


Total unique players in Grand Slams since 2010: 599


In [7]:
print(len(atp_recent))
print(len(filter_df_by_ids(atp_recent, all_unique_gs_players)))
print(len(get_gs_df(atp_recent)))

81831
64900
7493


## Data Dropping

In [8]:
atp_recent = atp_recent.drop(["winner_seed", "winner_entry", "loser_seed", "loser_entry"], axis=1)
print(f"Number of useless rows dropepd: {len(atp_recent[atp_recent['score'] == 'W/O'])}")
atp_recent = atp_recent[atp_recent['score'] != 'W/O']
atp_recent = atp_recent.reset_index(drop=True)

Number of useless rows dropepd: 412


In [9]:
print(len(atp_recent))
print(len(filter_df_by_ids(atp_recent, all_unique_gs_players)))
print(len(get_gs_df(atp_recent)))

81419
64554
7462


In [10]:
check_nans(atp_recent)

Number of missing entries in 'surface': 53


Number of missing entries in 'winner_ht': 1603
Number of missing entries in 'winner_age': 5
Number of missing entries in 'loser_hand': 4
Number of missing entries in 'loser_ht': 3255
Number of missing entries in 'loser_age': 3
Number of missing entries in 'minutes': 8707
Number of missing entries in 'winner_rank': 698
Number of missing entries in 'winner_rank_points': 698
Number of missing entries in 'loser_rank': 1680
Number of missing entries in 'loser_rank_points': 1683
Number of rows with missing player statistics: 6866
Total rows with at least one missing value: 9841


In [11]:
check_nans(filter_df_by_ids(atp_recent, all_unique_gs_players))

Number of missing entries in 'surface': 9
Number of missing entries in 'winner_ht': 112
Number of missing entries in 'loser_hand': 1
Number of missing entries in 'loser_ht': 839
Number of missing entries in 'minutes': 5124
Number of missing entries in 'winner_rank': 69
Number of missing entries in 'winner_rank_points': 69
Number of missing entries in 'loser_rank': 426
Number of missing entries in 'loser_rank_points': 426
Number of rows with missing player statistics: 3407
Total rows with at least one missing value: 5625


In [12]:
check_nans(get_gs_df(atp_recent))

Number of missing entries in 'minutes': 496
Number of missing entries in 'winner_rank': 2
Number of missing entries in 'winner_rank_points': 2
Number of missing entries in 'loser_rank': 16
Number of missing entries in 'loser_rank_points': 16
Number of rows with missing player statistics: 1
Total rows with at least one missing value: 515


In [13]:
MAX_RANK = 2000
HEIGHT_MEAN = (
    atp_recent[["winner_ht", "loser_ht"]]
    .stack()                 
    .mean()
)

atp_recent = atp_recent.dropna(subset=[
    "w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced",
    "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced",
    ]
)
atp_recent = atp_recent.reset_index(drop=True)

for side in ["winner", "loser"]:
    ht_col   = f"{side}_ht"
    rank_col  = f"{side}_rank"
    pts_col   = f"{side}_rank_points"

    atp_recent[rank_col]  = atp_recent[rank_col].fillna(MAX_RANK)
    atp_recent[pts_col]   = atp_recent[pts_col].fillna(0)
    atp_recent[ht_col] = atp_recent[ht_col].mask(
    atp_recent[ht_col].isna() | (atp_recent[ht_col] < 150),
    HEIGHT_MEAN
    )

In [14]:
check_nans(get_gs_df(atp_recent))

Number of missing entries in 'minutes': 496
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 496


In [15]:
check_nans(atp_recent)

Number of missing entries in 'winner_age': 1
Number of missing entries in 'loser_age': 1
Number of missing entries in 'minutes': 1854
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 1856


In [16]:
check_nans(filter_df_by_ids(atp_recent, all_unique_gs_players))

Number of missing entries in 'minutes': 1728
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 1728


## Parse score

In [17]:
score_features = atp_recent['score'].apply(parse_score).apply(pd.Series)

atp_recent = pd.concat([atp_recent, score_features], axis=1)
atp_recent['minutes'] = atp_recent.apply(
    lambda row: row['total_games'] * 6 if pd.isna(row['minutes']) else row['minutes'],
    axis=1
)

In [18]:
check_nans(atp_recent)
check_nans(get_gs_df(atp_recent))
check_nans(filter_df_by_ids(atp_recent, all_unique_gs_players))


Number of missing entries in 'winner_age': 1
Number of missing entries in 'loser_age': 1
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 2
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 0
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 0


In [19]:
check_nans(atp_recent)
check_nans(get_gs_df(atp_recent))
check_nans(filter_df_by_ids(atp_recent, all_unique_gs_players))

Number of missing entries in 'winner_age': 1
Number of missing entries in 'loser_age': 1
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 2
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 0
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 0


In [20]:
display(atp_recent[atp_recent['winner_age'].isna()])
display(atp_recent[atp_recent['loser_age'].isna()])

mask1 = (atp_recent["winner_id"] == 209294) & (atp_recent["winner_age"].isna()) 
atp_recent.loc[mask1, "winner_age"] = 20

mask2 = (atp_recent["loser_id"] == 127195) & (atp_recent["loser_age"].isna()) 
atp_recent.loc[mask2, "loser_age"] = 31

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,total_games,num_tiebreaks,has_tiebreak,winner_tb_won,loser_tb_won
74523,2024-M-DC-2024-WG2-PO-ESA-POC-01,Davis Cup WG2 PO: ESA vs POC,Hard,4,D,2024-02-02,1,209294,Diego Duran,U,185.855843,ESA,NaN,210249,Gillian Osmont,R,185.855843,FRA,20.7,6-3 6-0,3,RR,69.0,4.0,1.0,46.0,27.0,19.0,13.0,8.0,0.0,1.0,2.0,5.0,46.0,24.0,16.0,3.0,7.0,4.0,9.0,2000.0,0.0,2000.0,0.0,15,0,False,0,0


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,total_games,num_tiebreaks,has_tiebreak,winner_tb_won,loser_tb_won
74522,2024-M-DC-2024-WG2-PO-CYP-MAR-01,Davis Cup WG2 PO: CYP vs MAR,Hard,4,D,2024-02-03,4,210308,Stylianos Christodoulou,U,185.855843,CYP,19.2,127195,Hamza Karmoussi,U,185.855843,MAR,NaN,6-3 6-1,3,RR,60.0,7.0,1.0,50.0,37.0,29.0,6.0,8.0,3.0,3.0,0.0,3.0,49.0,30.0,14.0,9.0,8.0,2.0,6.0,1114.0,8.0,2000.0,0.0,16,0,False,0,0


In [21]:
check_nans(atp_recent)
check_nans(get_gs_df(atp_recent))
check_nans(filter_df_by_ids(atp_recent, all_unique_gs_players))

Number of rows with missing player statistics: 0
Total rows with at least one missing value: 0
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 0
Number of rows with missing player statistics: 0
Total rows with at least one missing value: 0


In [22]:
print(len(atp_recent))
print(len(filter_df_by_ids(atp_recent, all_unique_gs_players)))
print(len(get_gs_df(atp_recent)))

74553
61147
7461


## Match Date

In [23]:
OFFSET_G = {
    "R128": 0,  "R64": 2,  "R32": 4,  "R16": 6,
    "QF": 9,    "SF": 11,  "F": 13,
    "BR": 0,    "RR": 0,  "ER": 0
}

OFFSET_O = {
    "R128": 0, "R64": 0,  "R32": 1,  "R16": 2,
    "QF": 3,   "SF": 4,   "F": 6,
    "BR": 5,   "RR": 0,   "ER": 0
}

OFFSET_AFM = {
    "R128": 0, "R64": 1,  "R32": 2,  "R16": 3,
    "QF": 4,   "SF": 5,   "F": 7,
    "BR": 6,   "RR": 0,   "ER": -1
}

OFFSET_D = {
    "R128": 0, "R64": 0,  "R32": 0,  "R16": 0,
    "QF": 0,   "SF": 0,   "F": 0,
    "BR": 0,   "RR": 0,   "ER": 0
}

level_offset_map = {
    "G": OFFSET_G,
    "O": OFFSET_O,
    "A": OFFSET_AFM,   
    "F": OFFSET_AFM,   
    "M": OFFSET_AFM,   
    "D": OFFSET_D      
}

atp_recent["offset_days"] = atp_recent.apply(
    lambda r: level_offset_map[r["tourney_level"]].get(r["round"], 0),
    axis=1
)


atp_recent["match_date"] = (
    atp_recent["tourney_date"]
    + pd.to_timedelta(atp_recent["offset_days"], unit="D")
)

## Some Regularization

In [24]:
atp_recent['tourney_name'] = atp_recent['tourney_name'].replace('Us Open', 'US Open')
atp_recent['winner_hand'] = atp_recent['winner_hand'].replace(['U', 'A'], 'R')
atp_recent['loser_hand'] = atp_recent['loser_hand'].replace(['U', 'A'], 'R')

## Symmetric Recording of Players

In [25]:
winner_loser_pairs = {
    'winner_id'             : 'loser_id',
    'winner_name'           : 'loser_name',
    'winner_hand'           : 'loser_hand',
    'winner_ht'             : 'loser_ht',
    'winner_ioc'            : 'loser_ioc',
    'winner_age'            : 'loser_age',
    'w_ace'                 : 'l_ace',
    'w_df'                  : 'l_df',
    'w_svpt'                : 'l_svpt',
    'w_1stIn'               : 'l_1stIn',
    'w_1stWon'              : 'l_1stWon',
    'w_2ndWon'              : 'l_2ndWon',
    'w_SvGms'               : 'l_SvGms',
    'w_bpSaved'             : 'l_bpSaved',
    'w_bpFaced'             : 'l_bpFaced',
    'winner_rank'           : 'loser_rank',
    'winner_rank_points'    : 'loser_rank_points',
    'winner_tb_won'         : 'loser_tb_won'
}

rng = np.random.default_rng(42)       
random_mask = rng.integers(2, size=len(atp_recent))

for w_col, l_col in winner_loser_pairs.items():
    
    base_name = w_col.replace('winner_', '').replace('loser_', '') \
                     .replace('w_', '').replace('l_', '')
    
    p1_col = f'p1_{base_name}'
    p2_col = f'p2_{base_name}'
    
    atp_recent[p1_col] = np.where(random_mask == 0, atp_recent[w_col], atp_recent[l_col])
    atp_recent[p2_col] = np.where(random_mask == 0, atp_recent[l_col], atp_recent[w_col])

atp_recent['result'] = np.where(random_mask == 0, 1, 0)

atp_recent.drop(columns=list(winner_loser_pairs.keys()) + list(winner_loser_pairs.values()), 
        inplace=True, errors='ignore')

atp_recent.to_csv("../data/processed/Preprocessed_Data.csv", index=False)
